In [1]:
# Import the elementary libraries
import pandas as pd
import numpy as np
import datetime

In [8]:
# We proceed to load the CSV files (Dataset)
amazon_df = pd.read_csv("./datasets/amazon_prime_titles.csv")
disney_df = pd.read_csv("./datasets/disney_plus_titles.csv")
hulu_df = pd.read_csv("./datasets/hulu_titles.csv")
netflix_df = pd.read_csv("./datasets/netflix_titles.csv")

In [ ]:
# We quickly review how the df is composed. We know that all 4 have the same structure)
# So we will review only 1 df
amazon_df.head(3)

In [9]:
# Here we will assign the "id" column, which consists of the 1st letter of the platform and the "show_id"
amazon_df["id"] = "a" + amazon_df["show_id"]
disney_df["id"] = "d" + disney_df["show_id"]
hulu_df["id"] = "h" + hulu_df["show_id"]
netflix_df["id"] = "n" + netflix_df["show_id"]

In [ ]:
# We verify that the "id" column has been created correctly
amazon_df.head(3)

In [10]:
# We proceed to join the 4 df in one to make the necessary changes
platform_movies = pd.concat([amazon_df, disney_df, hulu_df, netflix_df], axis=0)

# We proceed to fill in the null values ​​of the "rating" column, with the letter "G" for General for all audience.
platform_movies["rating"].fillna("G", inplace=True)

In [ ]:
# Verify that the nulls of the "rating" column were filled correctly
platform_movies.head(3)

In [11]:
# Here we will change the date format "date_added" to YYYY-mm-dd 
platform_movies["date_added"] = pd.to_datetime(platform_movies["date_added"])

In [ ]:
# We verify that the changes in the date format were made correctly
platform_movies.head(3)

In [12]:
# We proceed to change all the text fields to lowercase
platform_movies = platform_movies.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

In [ ]:
# We verify that all the fields were converted to lowercase
platform_movies.head(3)

In [13]:
# We proceed to separate the "duration" column into 2 columns ("duration_int and duration_type").
# We must consider that "platform_movies" contains nulls in the "duration" column, so when we want to convert the
# Numeric part null to "int", we will have a problem. To do this, We will treat the data in the following way ...
platform_movies[["duration_int", "duration_type"]] = platform_movies["duration"].str.extract(r'(\d+)\s*(\D+)', expand=True)
platform_movies['duration_int'] = platform_movies['duration_int'].astype('Int64')

# Finally, we remove the "duration" column
platform_movies.drop("duration",inplace=True,axis=1)

In [ ]:
# Verify that the "duration" field was separated into 2 columns correctly (int and type)
platform_movies.head(3)

In [7]:
# We proceed to load and transform the files in the "ratings" folder
# First step: Upload the csv files.
rating_1 = pd.read_csv("./datasets/ratings/1.csv")
rating_2 = pd.read_csv("./datasets/ratings/2.csv")
rating_3 = pd.read_csv("./datasets/ratings/3.csv")
rating_4 = pd.read_csv("./datasets/ratings/4.csv")
rating_5 = pd.read_csv("./datasets/ratings/5.csv")
rating_6 = pd.read_csv("./datasets/ratings/6.csv")
rating_7 = pd.read_csv("./datasets/ratings/7.csv")
rating_8 = pd.read_csv("./datasets/ratings/8.csv")

# Then, we proceed to concatenate them into one.
ratings = pd.concat([rating_1, rating_2, rating_3, rating_4, rating_5, rating_6, rating_7, rating_8], axis=0)

In [8]:
# We check the dataset looking for duplicates
ratings.duplicated().sum()

124

In [9]:
# We perform transformations on the df ratings
# We will eliminate the duplicates, because it's impossible for the same user make more than 1 review
# For the same movie at the same time. He should be able to make just one review per movie.
ratings.drop_duplicates(inplace=True)

# After that,We'll change the name of the column "rating" for "scores"
ratings.rename(columns={"rating": "scores"}, inplace=True)

# The next step will be to change the format of timestamp to YYYY-MM-DD  
ratings["timestamp"] = ratings["timestamp"].apply(lambda d: datetime.datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))


In [10]:
# We check if all the changes were made correctly.
ratings

,userId,scores,timestamp,movieId
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186
2,1,5.0,2015-03-09,hs2381
3,1,5.0,2015-03-09,ns3663
4,1,5.0,2015-03-09,as9500
...,...,...,...,...
1499995,124380,4.5,2007-12-04,ns5272
1499996,124380,2.5,2007-12-04,ns5492
1499997,124380,3.5,2007-12-04,hs305
1499998,124380,4.5,2007-12-04,ns7881


In [6]:
# Save the df with all the changes made in a "csv".
ratings.to_csv("./datasets/scores.csv",index=False)

In [6]:
# Here We calculate the "mean score" for each movieId and keep them at a new df.
prom_scores_df = ratings.groupby("movieId")["scores"].mean().to_frame().reset_index()

# After that, We round the  scores to 2 decimals
prom_scores_df["scores"] = round(prom_scores_df["scores"],2)

# Now We change the name of the column "scores" for "prom_scores"
prom_scores_df.rename(columns = {"scores":"prom_scores"},inplace=True)

In [7]:
# We check that all changes made on "prom_scores_df" were made correctly.
prom_scores_df

,movieId,prom_scores
0,as1,3.47
1,as10,3.44
2,as100,3.61
3,as1000,3.56
4,as1001,3.59
...,...,...
22993,ns995,3.52
22994,ns996,3.63
22995,ns997,3.53
22996,ns998,3.58


In [14]:
# The next step will be to merge "prom_scores_df" to our main df "platform_movies"
platform_movies = platform_movies.merge(prom_scores_df, left_on="id", right_on="movieId")

# And We check if the merged was made correctly.
platform_movies

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,id,duration_int,duration_type,movieId,prom_scores
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,as1,113,min,as1,3.47
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,as2,110,min,as2,3.55
2,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,74,min,as3,3.50
3,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",as4,69,min,as4,3.54
4,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,45,min,as5,3.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,s8803,movie,zodiac,david fincher,"mark ruffalo, jake gyllenhaal, robert downey j...",united states,2019-11-20,2007,r,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",ns8803,158,min,ns8803,3.44
22994,s8804,tv show,zombie dumb,NaN,NaN,NaN,2019-07-01,2018,tv-y7,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",ns8804,2,seasons,ns8804,3.52
22995,s8805,movie,zombieland,ruben fleischer,"jesse eisenberg, woody harrelson, emma stone, ...",united states,2019-11-01,2009,r,"comedies, horror movies",looking to survive in a world taken over by zo...,ns8805,88,min,ns8805,3.42
22996,s8806,movie,zoom,peter hewitt,"tim allen, courteney cox, chevy chase, kate ma...",united states,2020-01-11,2006,pg,"children & family movies, comedies","dragged from civilian life, a former superhero...",ns8806,88,min,ns8806,3.59


In [15]:
# Now, We will drop the column "movieId", We dont need that col, because we already have the col "id"
platform_movies.drop("movieId",inplace=True,axis=1)

# We check the changes
platform_movies

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,id,duration_int,duration_type,prom_scores
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,as1,113,min,3.47
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,as2,110,min,3.55
2,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,74,min,3.50
3,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",as4,69,min,3.54
4,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,45,min,3.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,s8803,movie,zodiac,david fincher,"mark ruffalo, jake gyllenhaal, robert downey j...",united states,2019-11-20,2007,r,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",ns8803,158,min,3.44
22994,s8804,tv show,zombie dumb,NaN,NaN,NaN,2019-07-01,2018,tv-y7,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",ns8804,2,seasons,3.52
22995,s8805,movie,zombieland,ruben fleischer,"jesse eisenberg, woody harrelson, emma stone, ...",united states,2019-11-01,2009,r,"comedies, horror movies",looking to survive in a world taken over by zo...,ns8805,88,min,3.42
22996,s8806,movie,zoom,peter hewitt,"tim allen, courteney cox, chevy chase, kate ma...",united states,2020-01-11,2006,pg,"children & family movies, comedies","dragged from civilian life, a former superhero...",ns8806,88,min,3.59


In [16]:
# And finally, we save the df with all the changed made in a "csv".
platform_movies.to_csv("./datasets/platform_movies_scores.csv",index=False)